In [1]:
#Import necessary libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import numpy as np
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Agis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Agis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Agis\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#Load the dataset and document pickle files (for response processing in chatbot framework)
from keras.models import load_model
model = load_model('NurseBot_model')
import json
import pickle
intents = json.loads(open('intents.json', errors = "ignore").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [3]:
#Create Chatbot Framework in Response Processing
def clean_up_msg(message):
    # ignore punctuation from words
    ignore_words = ['?',',','!','.']
    tokenized_words = nltk.word_tokenize(message)
    clean_words = [lemmatizer.lemmatize(w.lower()) for w in tokenized_words if w not in ignore_words]
    return clean_words

def bow(message, words):
    clean_words = clean_up_msg(message)
    bag = [0] * len(words)
    for clean_word in clean_words:
        for idx, word in enumerate(words):
            if word == clean_word:
                bag[idx] = 1
    return np.array(bag)

def predict_class(message):
    bag_of_words = bow(message, words)
    res = model.predict(np.array([bag_of_words]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[idx, result] for idx, result in enumerate(res) if result > ERROR_THRESHOLD]
    # Sort by Probability
    results.sort(key= lambda x: x[1], reverse= True)
    intent_with_probability = [{'intent': classes[res[0]], 'probabilty' : str(res[1])} for res in results]
    
    return intent_with_probability

def getResponse(ints_with_probability):
    if ints_with_probability and ints_with_probability[0]['probabilty']:
        tag = ints_with_probability[0]['intent']
        list_of_intents = intents['intents']

        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result

def chatbot_response(msg):
    ints_with_probability = predict_class(msg)
    res = getResponse(ints_with_probability)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="white", font=("Verdana", 10 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "NurseBot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("NurseBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="#17202A", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",10,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#2C3E50", fg='#ffffff',width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
